In [1]:
import sys
sys.path.append('../')

In [1]:
# %%capture
%run common.ipynb

# importnb allows us to load the code from other notebooks
# from importnb import Notebook
# with Notebook():
#     import OfficeAutomation.common as common  # will probably become OfficeAutomation.common

In [13]:
import crestdsl.model as crest
import crestdsl.simulation as crestsim
import crestdsl.ui as ui

In [14]:
@crest.dependency(source="req_electricity_out", target="switch_in")
class Lamp(ElectricalDevice, crest.Entity):
    switch_in = crest.Input(res_onoff_switch, value="off") # on/off
    off = current = crest.State()
    on = crest.State()
    need_watt = crest.Local(res_electricity, value=100)

    turn_on = crest.Transition(source=off, target=on, guard=(lambda self: self.switch_in.value == "on" and self.electricity_in.value >= self.need_watt.value))
    turn_off = crest.Transition(source=on, target=off, guard=(lambda self: self.switch_in.value == "off" or self.electricity_in.value < self.need_watt.value))
    
    @crest.update(state=[on,off], target="req_electricity_out")
    def update_req_on_off(self, dt):
        if self.switch_in.value == "on":
            return self.need_watt.value
        else:
            return 0

In [15]:
class FragileLamp(Lamp, crest.Entity):
    switch_in = crest.Input(res_onoff_switch, value="off") # on/off
    spike_timeout = crest.Local(res_time, value=0.0015)  # time unit = minutes hence 0.0015 is roughly .1 seconds
    
    spike = crest.State()
    broken = crest.State()
    
    to_spike = crest.Transition(source="off", target=spike, guard=(lambda self: self.switch_in.value == "on" and self.electricity_in.value >= self.need_watt.value))
    to_broken = crest.Transition(source=spike, target=broken, guard=(lambda self: self.spike_timeout.value <= 0))
    
 
    @crest.update(state=spike, target=spike_timeout)
    def reduce_spike_timeout(self, dt):
        return self.spike_timeout.value - dt
    
    @crest.update(state=spike, target="req_electricity_out")
    def update_req_spike(self, dt):
        return 1000

    @crest.update(state=broken, target="req_electricity_out")
    def update_req_broken(self, dt):
        return 0
    
# crest.Validator(FragileLamp()).check_all()

In [16]:
@crest.nodependencies
class EmergencyLamp(ElectricalDevice, crest.Entity):
    spike_timeout = crest.Local(res_time, value=0.0015)  # time unit = minutes hence 0.0015 is roughly .1 seconds
    need_watt = crest.Local(res_electricity, value=100)
    
    off = current = crest.State()
    on = crest.State()
    spike = crest.State()
    broken = crest.State()
    
    turn_on = crest.Transition(source=off, target=on, guard=(lambda self: self.electricity_in.value >= self.need_watt.value))
    turn_off = crest.Transition(source=on, target=off, guard=(lambda self: self.electricity_in.value < self.need_watt.value))
    
    to_spike = crest.Transition(source=off, target=spike, guard=(lambda self: self.electricity_in.value >= self.need_watt.value))
    to_broken = crest.Transition(source=spike, target=broken, guard=(lambda self: self.spike_timeout.value <= 0))
    
    @crest.update(state=spike, target=spike_timeout)
    def reduce_spike_timeout(self, dt):
        return self.spike_timeout.value - dt
    
    @crest.update(state=spike, target="req_electricity_out")
    def set_spike_electricity(self, dt):
        return 1000
    
    @crest.update(state=off, target="req_electricity_out")
    def update_req_off(self, dt):
        return 0
    
    @crest.update(state=on, target="req_electricity_out")
    def update_req_on(self, dt):
        return self.need_watt.value

# ui.plot(EmergencyLamp())
# crest.Validator(EmergencyLamp()).check_all()